# 1. Load packages

In [1]:
from analysis_tools.common import *
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import sklearn
import cv2
import os

%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
sklearn.random.seed(RANDOM_STATE)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


# 2. Load dataset

In [2]:
train_full_data_meta = pd.read_csv(join(PATH.input, 'train_df.csv'), index_col=0)
test_data_meta       = pd.read_csv(join(PATH.input, 'test_df.csv'), index_col=0)

with ProgressBar():
    X_train_full = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.train)])
    X_test       = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.test)])
y_train_full = train_full_data_meta[['label']]
    
print("- Number of train full data:", len(X_train_full))
print("- Number of test data:", len(X_test))

[########################################] | 100% Completed |  7.5s
[########################################] | 100% Completed |  3.6s
- Number of train full data: 4277
- Number of test data: 2154


# 3. Training

In [3]:
from sklearn.preprocessing import OneHotEncoder

IMG_SIZE    = 512
input_shape = (IMG_SIZE, IMG_SIZE, 3)

with ProgressBar():
    X_train_full = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_train_full]))
    X_test       = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_test]))
y_enc = OneHotEncoder(sparse=False, dtype=bool)
y_train_full = y_enc.fit_transform(y_train_full)
n_classes = len(y_enc.categories_[0])
print("- Number of classes:", n_classes)

[########################################] | 100% Completed |  1.9s
[########################################] | 100% Completed |  0.8s
- Number of classes: 88


In [4]:
aug_model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

def preprocess(ds, training, batch_size, augment=True):
    ds = ds.cache().batch(batch_size)
    if training:
        ds = ds.shuffle(buffer_size=1000).prefetch(tf.data.AUTOTUNE)
        if augment:
            ds = ds.map(lambda X, y, sw: (aug_model(X), y, sw), num_parallel_calls=tf.data.AUTOTUNE)
    return ds

# fig, axes = plt.subplots(5, 15, figsize=(30, 10))
# for row, ax_cols in enumerate(axes):
#     for ax in ax_cols:
#         ax.imshow(aug_model(X_train_full[row]))
#         ax.axis('off')
# plt.show()

## 3.1 Fix pretrained model

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, stratify=y_train_full)
sample_weight_train = compute_sample_weight(class_weight='balanced', y=y_train.argmax(1))
sample_weight_val   = compute_sample_weight(class_weight='balanced', y=y_val.argmax(1))

train_ds = preprocess(tf.data.Dataset.from_tensor_slices((X_train, y_train, sample_weight_train)), True, BATCH_SIZE)
val_ds   = preprocess(tf.data.Dataset.from_tensor_slices((X_val, y_val, sample_weight_val)), False, BATCH_SIZE)
test_ds  = preprocess(tf.data.Dataset.from_tensor_slices(X_test), False, BATCH_SIZE)

print("- train.shape:", X_train.shape, y_train.shape)
print("- val.shape:", X_val.shape, y_val.shape)
print("- test.shape:", X_test.shape)

- train.shape: (3207, 512, 512, 3) (3207, 88)
- val.shape: (1070, 512, 512, 3) (1070, 88)
- test.shape: (2154, 512, 512, 3)


In [6]:
from tensorflow_addons.metrics import F1Score

def build_model(n_classes, strategy):
    with strategy.scope():
        base_model = keras.applications.EfficientNetB0(include_top=False, input_shape=input_shape)
        base_model.trainable = False

        inputs  = keras.Input(input_shape)
        hidden  = base_model(inputs, training=False)
        hidden  = keras.layers.GlobalAveragePooling2D()(hidden)
        outputs = keras.layers.Dense(n_classes, activation='softmax')(hidden)
        model   = keras.Model(inputs, outputs)
        
        model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
    return model, base_model

In [7]:
from analysis_tools.modeling import *

model, base_model = build_model(n_classes, strategy)
history = model.fit(train_ds, validation_data=val_ds, epochs=1000, callbacks=get_callbacks(patience=20, plot_path=join(PATH.result, 'proposed3', 'fix_pretrained_model_nadam')))

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

101/101 [==============================] - 14s 111ms/step - loss: 0.4969 - f1_score: 0.7462 - val_loss: 1.0475 - val_f1_score: 0.5312
Epoch 44/1000
101/101 [==============================] - 13s 99ms/step - loss: 0.5170 - f1_score: 0.7184 - val_loss: 1.0617 - val_f1_score: 0.5195
Epoch 45/1000
101/101 [==============================] - 13s 98ms/step - loss: 0.4900 - f1_score: 0.7480 - val_loss: 1.0371 - val_f1_score: 0.4915
Epoch 46/1000
101/101 [==============================] - 12s 97ms/step - loss: 0.4816 - f1_score: 0.7442 - val_loss: 1.0367 - val_f1_score: 0.5453
Epoch 47/1000
101/101 [==============================] - 12s 96ms/step - loss: 0.4916 - f1_score: 0.7430 - val_loss: 1.0764 - val_f1_score: 0.5196
Epoch 48/1000
101/101 [==============================] - 12s 97ms/step - loss: 0.4887 - f1_score: 0.7526 - val_loss: 1.0061 - val_f1_score: 0.5557
Epoch 49/1000
101/101 [==============================] - 12s 99ms/step - loss: 0.4962 - f1_score: 0.7386 - val_loss: 1.0206 - val_f

## 3.2 Fine-tuning

In [8]:
with strategy.scope():
    base_model.trainable = True
    model.compile(optimizer=keras.optimizers.Nadam(2e-4), loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
history = model.fit(train_ds, validation_data=val_ds, epochs=1000, callbacks=get_callbacks(patience=20, plot_path=join(PATH.result, 'proposed3', 'fine_tuning_nadam')))

Epoch 1/1000
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1
101/101 [==============================] - 114s 532ms/step - loss: 1.4359 - f1_score: 0.4386 - val_loss: 1.4077 - val_f1_score: 0.4491
Epoch 2/1000
101/101 [==============================] - 39s 373ms/step - loss: 0.8188 - f1_score: 0.5809 - val_loss: 1.3228 - val_f1_score: 0.4638
Epoch 3/1000
101/101 [==============================] - 41s 370ms/step - loss: 0.8890 - f1_score: 0.5839 - val_loss: 1.4594 - val_f1_score: 0.5842
Epoch 4/1000
101/101 [==============================] - 39s 371ms/step - loss: 0.6605 - f1_score: 0.6483 - val_loss: 1.4837 - val_f1_score: 0.5361
Epoch 5/1000
101/101 [==============================] - 41s 371ms/step - loss: 0.5266 - f1_score: 0.7077 - val_loss: 1.2631 - val_f1_score: 0.5987
Epoch 6/1000
101/101 [==============================] - 39s 374ms/step - loss: 0.3852 - f1

101/101 [==============================] - 40s 369ms/step - loss: 0.0054 - f1_score: 0.9874 - val_loss: 0.7100 - val_f1_score: 0.8451
Epoch 53/1000
101/101 [==============================] - 41s 375ms/step - loss: 0.0184 - f1_score: 0.9841 - val_loss: 0.7911 - val_f1_score: 0.8332
Epoch 54/1000
101/101 [==============================] - 42s 385ms/step - loss: 0.0139 - f1_score: 0.9916 - val_loss: 0.9551 - val_f1_score: 0.8208
Epoch 55/1000
101/101 [==============================] - 41s 375ms/step - loss: 0.4175 - f1_score: 0.8408 - val_loss: 1.5033 - val_f1_score: 0.6455
Epoch 56/1000
101/101 [==============================] - 41s 371ms/step - loss: 0.3731 - f1_score: 0.8277 - val_loss: 1.1055 - val_f1_score: 0.6612
Epoch 57/1000
101/101 [==============================] - 40s 370ms/step - loss: 0.2101 - f1_score: 0.8637 - val_loss: 0.8305 - val_f1_score: 0.7349
Epoch 58/1000
101/101 [==============================] - 41s 374ms/step - loss: 0.1446 - f1_score: 0.9083 - val_loss: 0.7754 -

# 4. Evaluation

In [9]:
submission_file_path = join(PATH.output, 'proposed3_fine_tuning_nadam.csv')

pred_test = model.predict(test_ds)
submission = pd.read_csv(join(PATH.input, 'sample_submission.csv'), index_col=0)
submission['label'] = y_enc.inverse_transform(pred_test)
submission.to_csv(submission_file_path)
submission

,label
index,
0,tile-glue_strip
1,grid-good
2,transistor-good
3,tile-gray_stroke
4,tile-good
...,...
2149,tile-gray_stroke
2150,screw-good
2151,grid-good


# 5. Submission

In [10]:
from dacon_submit_api.dacon_submit_api import post_submission_file

result = post_submission_file(
    submission_file_path,
    '137ff236e305f302819b930b3b5b72e948603f23c5249a516c32b536d5187a03', 
    '235894', 
    '어스름한 금요일 밤에', 
    'proposed3_fine_tuning_nadam'
)

{'isSubmitted': True, 'detail': 'Success'}
